In [1]:
import numpy as np
import pandas as pd

result = pd.read_csv('sample/results.csv', encoding='cp949')
result

,파일명,선종,상태,위치
0,1.jpg,범장망,표류,LAT N33.22.872 LON E124.5.074
1,2.jpg,범장망,이동,LAT N32.48.241 LON E124.38.553
2,3.jpg,범장망,조업,LAT N32.55.893 LON E124.33.731
3,4.jpg,타망,이동,LAT N35.40.891 LON E124.25.481
4,5.jpg,타망,이동,LAT N32.23.668 LON E125.33.053
5,6.jpg,타망,이동,LAT N32.59.352 LON E124.29.609
6,7.jpg,유망,조업,LAT N34.54.232 LON E124.42.394
7,8.jpg,유망,조업,LAT N34.41.829 LON E124.20.120
8,9.jpg,유망,표류,LAT N36.56.279 LON E124.39.427
9,10.jpg,안강망,조업,LAT N34.15.543 LON E125.41.175


In [2]:
from PIL import Image
import numpy as np
import easyocr
import os

# EasyOCR Reader 초기화 (한국어 및 영어 설정 가능)
reader = easyocr.Reader(['ko', 'en'], gpu=False)

# 이미지 파일이 저장된 디렉터리 경로
image_dir = '테스트셋'  # 이미지들이 저장된 디렉터리

# 디렉터리 내 모든 파일에 대해 반복
for image_file in os.listdir(image_dir):
    # 이미지 파일 경로
    image_path = os.path.join(image_dir, image_file)
    
    try:
        # PIL을 사용해 이미지 불러오기
        with Image.open(image_path) as img:
            # 이미지 크기 정보 가져오기
            width, height = img.size

            # 하단 5% 부분 잘라내기
            start_row = int(height * 0.95)  # 하단 5%의 시작 좌표 계산
            cropped_img = img.crop((0, start_row, width, height))  # 하단 5% 영역 자르기

            # 자른 이미지를 numpy 배열로 변환
            cropped_image_np = np.array(cropped_img)
    
    except Exception as e:
        print(f"Skipping {image_file}, error: {e}")
        continue

    # EasyOCR로 텍스트 추출
    result = reader.readtext(cropped_image_np, beamWidth=10, text_threshold=0.7)

    # 이미지 파일 이름 출력
    print(f"\nExtracting text from {image_file} (bottom 5%):")

    # 추출된 텍스트를 한 줄로 연결
    detected_text = ' '.join([text for (bbox, text, prob) in result])

    # 한 줄로 출력
    print(f"Detected Text: {detected_text}")


Using CPU. Note: This module is much faster with a GPU.



Extracting text from 1.jpg (bottom 5%):
Detected Text: LLar기 15 IUU5 TLJnt 15 + Ih "AII [i 느H77: J1HITI In- `ei HrrLnrert-V 개

Extracting text from 10.jpg (bottom 5%):
Detected Text: LLary 32- It'iLDnt I 1J.bb1' A Ui:kn: 4-minsKa Hdrurcert-UJJ

Extracting text from 11.jpg (bottom 5%):
Detected Text: 16U -0 +90 +180

Extracting text from 12.jpg (bottom 5%):
Detected Text: TLat N 340 52.795' TLon E 1240 5.796' Alt Of S Rng: 4075m Ins Nav HdgUncert=0.09

Extracting text from 13.jpg (bottom 5%):
Detected Text: TLat N 349 9.179' TLon E 1259 56.149 AIt Or $ Rng: 1013Im Ins Nav HdgUncert=0 10

Extracting text from 14.jpg (bottom 5%):
Detected Text: Lat: N 328 10,624Lan: E 1258 14.432 AZ: 3/8 -10,28 Ua-Jun-2019 15:33.35L

Extracting text from 15.jpg (bottom 5%):
Detected Text: TLat N 09 0.000' TLon E 09 0.000' Alt Of $ Rng: Om Ins

Extracting text from 16.jpg (bottom 5%):
Detected Text: 

Extracting text from 17.jpg (bottom 5%):
Detected Text: TLatN 349 52.670' TLon E 1259 7.083 AIt D S Rng: 

In [3]:
import os
import re
import pandas as pd
from PIL import Image
import numpy as np
import easyocr

# EasyOCR Reader 초기화 (한국어 및 영어 설정 가능)
reader = easyocr.Reader(['ko', 'en'], gpu=False)

# 이미지 파일이 저장된 디렉터리 경로
image_dir = '테스트셋'  # 이미지들이 저장된 디렉터리

# 좌표 추출 및 변환 함수
def extract_and_format_coordinates(text):
    # 패턴: "TLat N 340 11.916' TLon E 1240 26.179'" 또는 "Lat: N 329 10.035 Lan: E 1269 2.611"
    pattern = r"(?:TLat|Lat[:\s]?)\s?([NS])\s?(\d{2,3})[,\s\"\']?\s?(\d+[\.,]?\d*)'?\s?(?:TLon|Lan[:\s]?)\s?([EW])\s?(\d{3,4})[,\s\"\']?\s?(\d+[\.,]?\d*)'?"

    # 콤마를 점으로 변환 (숫자 처리 시 일관성 유지)
    text = text.replace(',', '.')

    match = re.search(pattern, text)

    if match:
        # 정규식 그룹 추출: N/S, 위도 각도 (첫 두 자리), 위도 분, E/W, 경도 각도 (첫 세 자리), 경도 분
        lat_dir = match.group(1)
        lat_deg = match.group(2)[:2]  # 위도는 두 자리만 유지
        lat_min = match.group(3).replace(',', '.')
        lon_dir = match.group(4)
        lon_deg = match.group(5)[:3]  # 경도는 세 자리만 유지
        lon_min = match.group(6).replace(',', '.')

        # 원하는 형식으로 좌표를 변환 (LAT N33.22.872 LON E124.5.074)
        formatted_coords = f"LAT {lat_dir}{lat_deg}.{lat_min} LON {lon_dir}{lon_deg}.{lon_min}"
        return formatted_coords

    return None

# 파일명과 좌표 데이터를 저장할 리스트
data = []

# 파일 목록을 번호 순으로 정렬하여 순차적으로 읽기
for image_file in sorted(os.listdir(image_dir), key=lambda x: int(re.search(r'\d+', x).group())):
    # 이미지 파일 경로
    image_path = os.path.join(image_dir, image_file)
    
    # 이미지 파일 확장자 확인
    if not image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue  # 이미지 파일이 아닌 경우 스킵
    
    try:
        # PIL을 사용해 이미지 불러오기
        with Image.open(image_path) as img:
            # 이미지 크기 정보 가져오기
            width, height = img.size

            # 하단 5% 부분 잘라내기
            start_row = int(height * 0.95)  # 하단 5%의 시작 좌표 계산
            cropped_img = img.crop((0, start_row, width, height))  # 하단 5% 영역 자르기

            # 자른 이미지를 numpy 배열로 변환
            cropped_image_np = np.array(cropped_img)
    
    except Exception as e:
        print(f"Skipping {image_file}, error: {e}")
        continue

    # EasyOCR로 텍스트 추출
    result = reader.readtext(cropped_image_np, beamWidth=10, text_threshold=0.7)

    # 추출된 텍스트를 한 줄로 연결
    detected_text = ' '.join([text for (bbox, text, prob) in result])

    # 좌표 추출 및 변환 시도
    formatted_coordinates = extract_and_format_coordinates(detected_text)

    # 데이터 추가 (파일명, 좌표)
    data.append([image_file, formatted_coordinates if formatted_coordinates else "No valid coordinates"])

# DataFrame 생성 및 CSV 저장
df = pd.DataFrame(data, columns=['파일명', '좌표']).set_index('파일명')
df.to_csv('추출된_좌표.csv', encoding='utf-8-sig')
print("CSV 파일로 저장되었습니다: 추출된_좌표.csv")

Using CPU. Note: This module is much faster with a GPU.


CSV 파일로 저장되었습니다: 추출된_좌표.csv


In [4]:
df=pd.read_csv('추출된_좌표.csv')
df

,파일명,좌표
0,1.jpg,No valid coordinates
1,2.jpg,No valid coordinates
2,3.jpg,No valid coordinates
3,4.jpg,No valid coordinates
4,5.jpg,LAT N32.56.001 LON E126.9.254
5,6.jpg,No valid coordinates
6,7.jpg,No valid coordinates
7,8.jpg,No valid coordinates
8,9.jpg,No valid coordinates
9,10.jpg,No valid coordinates
